# 8. MagicO

`PyLapi` resource data functionalities are implemented using ta utility class, `MagicO` under the hood, to enable the resource object's `.data` attribute notation and the resource object itself to be subscriptable using a dynamic path.

In this tutorial, we explain `MagicO` in detail and how to use them efficiently.

---
Let us define the following `dict` object as an example.

In [1]:
my_dict = {
    "a": 1,
    "b": {
        "c": 3,
        "d": 4
    },
    "e": [
        {"f": 6},
        "xyz"
    ]
}

To access the attribute "f", you need to use a series of subscripts, like this:

In [2]:
print(my_dict["e"][0]["f"])

6


## MagicO

As a programmer, you would find it more natural to access a dict object attribute using syntax like this: `my_dict.e[0].f`. This is what `MagicO` can do for you.

In [3]:
from pylapi import MagicO

my_magic = MagicO(my_dict)
print(my_magic.e[0].f)

6


You may create new attributes, change them, and delete them using the attribute notation.

In [4]:
print(my_magic) # original data
my_magic.b.g = 7
print(my_magic) # b.g is created
my_magic.b.g = 8
print(my_magic) # b.g is updated
del my_magic.b.g
print(my_magic) # b.g is deleted

{"a": 1, "b": {"c": 3, "d": 4}, "e": [{"f": 6}, "xyz"]}
{"a": 1, "b": {"c": 3, "d": 4, "g": 7}, "e": [{"f": 6}, "xyz"]}
{"a": 1, "b": {"c": 3, "d": 4, "g": 8}, "e": [{"f": 6}, "xyz"]}
{"a": 1, "b": {"c": 3, "d": 4}, "e": [{"f": 6}, "xyz"]}


Please note that in order to maintain the "attribute chain", each "non-leaf" node of an `MagicO` is itself an `MagicO` (while a leaf node is returned as is). But you can use the `MagicO.to_data()` method to obtain the underlying data object.

For example:

In [5]:
print(type(my_magic.e[0])) # <class 'pylapi.attr_dict.MagicO'>
print(type(my_magic.e[0].to_data())) # <class 'dict'>

<class 'pylapi.magico.MagicO'>
<class 'dict'>


`MagicO.to_data()` returns a pointer to the original data that you created the `MagicO` with. Updates to the pointer will affect the original data in `my_dict` as well.
Basically, `MagicO` is a wrapper for the original data you created it with.
They all share the same storage.

In [6]:
print(my_dict) # Original: {..., 'e': [{'f': 6}, 'xyz']}
my_attr_data = my_magic.to_data()

# Update the data object
my_attr_data["e"][1] = "abc"
print(my_dict) # Output: {..., 'e': [{'f': 6}, 'abc']}

# Update the MagicO object
my_magic.e[1] = "xyz"
print(my_dict) # Output: {..., 'e': [{'f': 6}, 'xyz']}

{'a': 1, 'b': {'c': 3, 'd': 4}, 'e': [{'f': 6}, 'xyz']}
{'a': 1, 'b': {'c': 3, 'd': 4}, 'e': [{'f': 6}, 'abc']}
{'a': 1, 'b': {'c': 3, 'd': 4}, 'e': [{'f': 6}, 'xyz']}


`MagicO` also works with list objects and supports all list behaviours. For example:

In [7]:
my_attr_list = MagicO(["a", {"b": 2}, "c"])
print(my_attr_list[-2].b) # 2
print(my_attr_list[1:]) # [{'b': 2}, 'c']
my_attr_list.append("d")
print(my_attr_list) # ['a', {'b': 2}, 'c', 'd']

2
[{'b': 2}, 'c']
['a', {'b': 2}, 'c', 'd']


In `PyLapi`, the resource object's `.data` attribute is an `MagicO`, so you can access the resource data using the attribute notation, e.g., `workspace.data.email_domains[1]`.

## MagicO

There are times when the attribute to access is dynamically determined, e.g., "$.e[0].f" as formulated by the code.
This is where a `MagicO` can be useful.

In [8]:
from pylapi import MagicO

my_magic = MagicO(my_dict)
print(my_magic["$.e[0].f"])

6


You may create new attributes, change them, and delete them as you would naturally do.

In [9]:
print(my_magic) # original data
my_magic["$.b.g"] = 7
print(my_magic) # g is created under b
my_magic["$.b.g"] = 8
print(my_magic) # g is updated
del my_magic["$.b.g"]
print(my_magic) # b.g is deleted

{"a": 1, "b": {"c": 3, "d": 4}, "e": [{"f": 6}, "xyz"]}
{"a": 1, "b": {"c": 3, "d": 4, "g": 7}, "e": [{"f": 6}, "xyz"]}
{"a": 1, "b": {"c": 3, "d": 4, "g": 8}, "e": [{"f": 6}, "xyz"]}
{"a": 1, "b": {"c": 3, "d": 4}, "e": [{"f": 6}, "xyz"]}


One feature unique to `MagicO` is deep attribute creation: If you assign a new sub-attribute, all missing parent attributes along the path will be created automatically. For example:

In [10]:
my_magic["$.b.g.h.i"] = 9
print(my_dict) # Attribute "b" is appended with "g.h" to get to "i"
del my_magic["$.b.g"] # Deleting the parent will delete its tree
print(my_dict) # Attribute "b.g" is deleted

{'a': 1, 'b': {'c': 3, 'd': 4, 'g': {'h': {'i': 9}}}, 'e': [{'f': 6}, 'xyz']}
{'a': 1, 'b': {'c': 3, 'd': 4}, 'e': [{'f': 6}, 'xyz']}


The symbol `$` represents the `.data` attribute, which can be omitted. In the above example, you may write `my_magic["e[0].f"]`. (IMHO, including the `$.` prefix is syntactically more pleasing.)

Unlike an MagicO, a `MagicO` resolves the node before returning, whether it is a leaf node or not.

In [11]:
print(type(my_magic["$.e[0]"])) # <class 'pylapi.magic_dict.MagicO'>
print(type(my_magic["$.e[0].f"])) # <class 'int'>

<class 'dict'>
<class 'int'>


For the `MagicO` itself, you can use the `MagicO.to_data()` to obtain the underlying object.

In [12]:
print(type(my_magic)) # <class 'pylapi.path_dict.MagicO'>
print(type(my_magic.to_data())) # <class 'dict'>

<class 'pylapi.magico.MagicO'>
<class 'dict'>


`MagicO.to_data()` returns a pointer to the original data that you created the `MagicO` with. Your updates will affect the original data `my_dict` as well.
Basically, `MagicO` is a wrapper to the original data you created it with.
They all share the same storage.

In [13]:
print(my_magic) # Original: {..., 'e': [{'f': 6}, 'xyz']}
my_path_data = my_magic.to_data()

# Update the data object
my_path_data["e"][1] = "abc"
print(my_dict) # Output: {..., 'e': [{'f': 6}, 'abc']}

# Update the MagicO object
my_magic["e[1]"] = "xyz"
print(my_dict) # Output: {..., 'e': [{'f': 6}, 'xyz']}

{"a": 1, "b": {"c": 3, "d": 4}, "e": [{"f": 6}, "xyz"]}
{'a': 1, 'b': {'c': 3, 'd': 4}, 'e': [{'f': 6}, 'abc']}
{'a': 1, 'b': {'c': 3, 'd': 4}, 'e': [{'f': 6}, 'xyz']}


`MagicO` works with list objects as well and supports all list behaviours. For example:

In [14]:
my_path_list = MagicO(["a", {"b": 2}, "c"])
print(my_path_list["$[-2].b"]) # 2
print(my_path_list["$[1:]"]) # [{'b': 2}, 'c']
my_path_list.append("d")
print(my_path_list) # ['a', {'b': 2}, 'c', 'd']

2
[{'b': 2}, 'c']
['a', {'b': 2}, 'c', 'd']


In `PyLapi`, the resource object itself "looks like" a `MagicO`, so you can access the resource data using the subscript notation, e.g., `workspace["$.email_domains[1]"]`.

## Manipulating resource objects "naturally"

Let us load some resource data and see how the `.data` attribute behaves like an `MagicO` and the resource object itself behaves like a `MagicO`.

First, let us quickly load some workspace resource data into a `workspace` resource object, then print to verify. (The string representation of a resource object is in a beautified JSON format.)

In [15]:
from aapi import aAPI
aAPI.auth(open(f"._asecret", "r").readlines()[0].strip())
workspace = aAPI.resource("workspace")
workspace.load(workspace.list()[0]["gid"])
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}


Since `workspace.data` is an `MagicO`, you can manipulate it using the attribute notation.

Also, the `@aAPI.resource_class` decorate has a resource mapping `gid="$.gid"` specified, so `workspace.gid` is mapped to `workspace.data.gid`. They share the same storage.

In [16]:
print(f"Workspace resource data:\n{workspace}")
print(f"workspace.data.gid: {workspace.data.gid}")
print(f"workspace.data.email_domains: {workspace.data.email_domains}")
print(f"workspace.gid: {workspace.gid}") # Mapped to workspace.data.gid

Workspace resource data:
{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}
workspace.data.gid: 1204597085072493
workspace.data.email_domains: ['accsoft.com.au']
workspace.gid: 1204597085072493


---
Please note that all changes illustrated in this tutorial happen locally in the resource object and will not affect any data entity stored on the backend.
To update the backend data, please see the [Search and Modify](./6.%20Search%20and%20Modify.ipynb) tutorial.

To add a new attribute to the resource data, you simply assign a value to the attribute under `workspace.data`.

In [17]:
workspace.data.new_attribute = "some value"
print(f"After 'new_attribute' is added, workspace becomes: {workspace}")

After 'new_attribute' is added, workspace becomes: {
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace",
  "new_attribute": "some value"
}


You can assign a new value to the attribute to update it.

In [18]:
workspace.data.new_attribute = "new value"
print(f"After 'new_attribute' is updated, workspace becomes: {workspace}")

After 'new_attribute' is updated, workspace becomes: {
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace",
  "new_attribute": "new value"
}


To delete an attribute, simply use the `del` command.

In [19]:
del workspace.data.new_attribute
print(f"After 'new_attribute' is deleted, workspace becomes: {workspace}")

After 'new_attribute' is deleted, workspace becomes: {
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}


You may also use `list` operations for list attributes. For example, you may append a new item to the `email_domains` list and modify list items individually.

In [20]:
workspace.data.email_domains.append("onmyweb.net")
print(f"After an email domain is appended, workspace.data.email_domains becomes: {workspace.data.email_domains}")

After an email domain is appended, workspace.data.email_domains becomes: ['accsoft.com.au', 'onmyweb.net']


In [21]:
workspace.data.email_domains[1] = "onmyweb.net.au"
print(f"After the second email domain is updated, workspace.data.email_domains becomes: {workspace.data.email_domains}")

After the second email domain is updated, workspace.data.email_domains becomes: ['accsoft.com.au', 'onmyweb.net.au']


You may use Python's slice and negative index as usual.

In [22]:
print(f"Last email domain: workspace.data.email_domains[-1]: {workspace.data.email_domains[-1]}")

Last email domain: workspace.data.email_domains[-1]: onmyweb.net.au


Deleting list items can be done in the same way.

In [23]:
del workspace.data.email_domains[-1]
print(f"\nAfter the last email domain is deleted, workspace becomes: {workspace.data}")


After the last email domain is deleted, workspace becomes: {"gid": "1204597085072493", "email_domains": ["accsoft.com.au"], "is_organization": true, "name": "accsoft.com.au", "resource_type": "workspace"}


## Subscriptable resource objects

Resource objects behave like a `MagicO` and therefore are subscriptable. This is useful if you want to programmatically determine which attribute to access.

In [24]:
print(workspace["$.gid"])
print(workspace["$.email_domains"])

1204597085072493
['accsoft.com.au']


Here the `$` symbol represents the resource data `workspace.data` and can be omitted.

In [25]:
# For simplicity, you may omit the "$." prefix in the subscript.
print(workspace["gid"])
print(workspace["email_domains"])

1204597085072493
['accsoft.com.au']


In [26]:
workspace["$.new_attribute"] = "some value"
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace",
  "new_attribute": "some value"
}


In [27]:
workspace["$.new_attribute"] = "new value"
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace",
  "new_attribute": "new value"
}


In [28]:
del workspace["$.new_attribute"]
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}


In [29]:
workspace["$.email_domains"].append("onmyweb.net")
print(workspace["email_domains"])

['accsoft.com.au', 'onmyweb.net']


In [30]:
workspace["$.email_domains[1]"] = "onmyweb.net.au"
print(workspace["email_domains"])

['accsoft.com.au', 'onmyweb.net.au']


In [31]:
print(workspace["$.email_domains[-1]"])

onmyweb.net.au


In [32]:
del workspace["$.email_domains[-1]"]
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}


## More on data types

The data type of the resource data depends on how you access them.
In most cases, you just use them naturally via the `.data` attribute or the path subscript,
but it is always good to be aware of the data types being passed around.

In [33]:
print(type(workspace)) # <class 'aapi.WorkspaceResource'>

print(type(workspace.data)) # <class 'pylapi.attr_dict.MagicO'>
print(type(workspace.data.email_domains)) # <class 'pylapi.attr_dict.MagicO'>
print(type(workspace.data.to_data())) # <class 'dict'>

print(type(workspace["$"])) # <class 'dict'>
print(type(workspace["$.email_domains"])) # <class 'list'>

<class 'aapi.WorkspaceResource'>
<class 'pylapi.magico.MagicO'>
<class 'pylapi.magico.MagicO'>
<class 'dict'>
<class 'dict'>
<class 'list'>


While a resource object behaves like a `MagicO`, it is *not* a `MagicO`, and it does *not* support `dict` and `list` operations.

By design, the subscriptable resource object is at arm's length from the internal resource data to protect it from accidental change.
If you want to obtain a pointer to the resource data, use `.resource_data` or its functional equivalent, `.data.to_data()`.

In [34]:
# Working on a deep copy of the resource data
workspace_data = workspace["$"]
workspace_data["is_organization"] = False # NOT updating the resource data
print(workspace["is_organization"]) # True
workspace["$.is_organization"] = False
print(workspace["$.is_organization"]) # False

# Working on a pointer of the resource data
workspace_data = workspace.resource_data
# Alternatively: workspace_data = workspace.data.to_data()
workspace.data.is_organization = True
print(workspace.data.is_organization) # True

True
False
True


---
This conclude the series of `PyLapi` tutorials. If you have any questions or experience any issues, please log a [PyLapi ticket on GitHub](https://github.com/jackyko8/pylapi/issues).

## End of page